<a href="https://colab.research.google.com/github/lucasdp5/ds2/blob/main/perea_ds2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import base64
from io import BytesIO
import datetime

In [2]:
Definimos una función para centralizar la lectura (Buenas prácticas de ingeniería)
def load_github_csv(file_name):
    # Reemplaza con tu base URL de GitHub Raw
    base_url = "https://raw.githubusercontent.com/lucasdp5/ds2/main/"
    full_url = f"{base_url}{file_name}"

    try:
        df = pd.read_csv(full_url)
        print(f"✅ {file_name} cargado exitosamente. Registros: {df.shape[0]}")
        return df
    except Exception as e:
        print(f"❌ Error al cargar {file_name}: {e}")
        return None

# Lectura de las tablas principales
df_reserva = load_github_csv("reserva.csv")
df_vehiculo = load_github_csv("vehiculo.csv")
df_cliente = load_github_csv("cliente.csv")
df_pago = load_github_csv("pago.csv")

✅ reserva.csv cargado exitosamente. Registros: 4036
✅ vehiculo.csv cargado exitosamente. Registros: 51
✅ cliente.csv cargado exitosamente. Registros: 7357
✅ pago.csv cargado exitosamente. Registros: 4060


In [3]:
df_reserva.columns


Index(['id', 'numero', 'sitio_salida', 'sitio_llegada', 'provincia_salida',
       'provincia_llegada', 'vehiculo_idvehiculo', 'fecha_salida',
       'fecha_devolucion', 'fecha_entrada', 'kms_entrada', 'kms_salida',
       'forma_de_pago', 'tarjeta_credito', 'numero_tarjeta',
       'fecha_venc_tarjeta', 'codigo_tarjeta', 'cliente_idcliente',
       'info_adicional', 'tarifa_1', 'km_tarifa_1', 'tarifa_2', 'km_tarifa_2',
       'tarifa_semanal', 'tarifa_mensual', 'cant_dias', 'meses', 'semanas',
       'subtotal', 'iva', 'importe_total', 'combustible_salida',
       'combustible_entrada', 'estado', 'created_by', 'updated_by',
       'created_at', 'updated_at', 'bonificacion', 'tipo', 'comision',
       'nrofactura', 'temporada', 'telcontacto', 'retorno', 'tanque_lleno',
       'franquicia_dano', 'franquicia_robo', 'combustible',
       'reducc_franq_danos_xdia', 'km_excedido', 'cargo_km_sup',
       'cargo_hs_sup', 'reduccion_franq_danos', 'km_origen_destino',
       'danios_devolucion'

In [4]:
#Se procede a eliminar todas aquellas columnas que no aportan valor
cols_reserva_ok = [
    'id', 'vehiculo_idvehiculo', 'cliente_idcliente', 'fecha_salida',
    'fecha_devolucion', 'fecha_entrada','created_at', 'total_dias', 'importe_total',
    'saldo', 'total_pagos', 'estado', 'subtotal',
    'kms_salida', 'kms_entrada', 'danios_devolucion'
]
df_reserva = df_reserva[cols_reserva_ok]

cols_vehiculo_ok = [
    'idvehiculo', 'marca', 'modelo', 'clase', 'kilometros',
    'propiedad', 'precio', 'km_aceite', 'venc_rto'
]
df_vehiculo = df_vehiculo[cols_vehiculo_ok]

cols_cliente_ok = ['idcliente', 'nombre', 'apellido', 'domicilio']
df_cliente = df_cliente[cols_cliente_ok]

cols_pago_ok = ['idpago', 'importe', 'fecha','reserva_id']
df_pago = df_pago[cols_pago_ok]

In [5]:
# 1. Conversión de Timestamps Unix a Datetime
# Si están en segundos, usamos unit='s'. Si están en milisegundos, unit='ms'.
unix_cols = {
    'df_reserva': ['fecha_salida', 'fecha_devolucion', 'fecha_entrada', 'created_at'],
    'df_vehiculo' : ['venc_rto'],
    'df_pago': ['fecha']
}

for col in unix_cols['df_reserva']:
    df_reserva[col] = pd.to_datetime(df_reserva[col], unit='s', errors='coerce')

for col in unix_cols['df_vehiculo']:
    df_vehiculo[col] = pd.to_datetime(df_vehiculo[col], unit='s', errors='coerce')

for col in unix_cols['df_pago']:
    df_pago[col] = pd.to_datetime(df_pago[col], unit='s', errors='coerce')

In [6]:
# Vamos a crear un df_master. Usaremos un left join partiendo de la tabla reserva,
# ya que es el hecho principal de nuestro negocio.

# 1. Unimos Reservas con Vehículos
# Queremos todas las reservas, y la info del vehículo si existe
df_master = pd.merge(
    df_reserva,
    df_vehiculo,
    left_on='vehiculo_idvehiculo',
    right_on='idvehiculo',
    how='left'
)

# 2. Unimos el resultado con Clientes
df_master = pd.merge(
    df_master,
    df_cliente,
    left_on='cliente_idcliente',
    right_on='idcliente',
    how='left'
)

# 3. Si quieres ver el comportamiento de pagos agregados
# Nota: Como una reserva puede tener muchos pagos, lo ideal es agrupar pagos primero
df_pago_agrupado = df_pago.groupby('reserva_id')['importe'].sum().reset_index()
df_pago_agrupado.columns = ['reserva_id', 'total_pagado_acumulado']

df_master = pd.merge(
    df_master,
    df_pago_agrupado,
    left_on='id',
    right_on='reserva_id',
    how='left'
)

print(f"Dataset Maestro creado. Forma final: {df_master.shape}")

Dataset Maestro creado. Forma final: (4036, 31)


In [7]:
df_master.columns

Index(['id', 'vehiculo_idvehiculo', 'cliente_idcliente', 'fecha_salida',
       'fecha_devolucion', 'fecha_entrada', 'created_at', 'total_dias',
       'importe_total', 'saldo', 'total_pagos', 'estado', 'subtotal',
       'kms_salida', 'kms_entrada', 'danios_devolucion', 'idvehiculo', 'marca',
       'modelo', 'clase', 'kilometros', 'propiedad', 'precio', 'km_aceite',
       'venc_rto', 'idcliente', 'nombre', 'apellido', 'domicilio',
       'reserva_id', 'total_pagado_acumulado'],
      dtype='object')

In [8]:
# 1. Definimos las columnas que realmente aportan valor al negocio
# Quitamos los IDs duplicados y nombres/apellidos si no vamos a hacer análisis de texto
cols_finales = [
    'id', 'fecha_salida', 'fecha_devolucion', 'fecha_entrada', 'created_at',
    'total_dias', 'importe_total', 'subtotal', 'saldo', 'total_pagos', 'total_pagado_acumulado',
    'estado', 'marca', 'modelo', 'clase',
    'kms_salida', 'kms_entrada', 'danios_devolucion',
    'kilometros', 'km_aceite', 'venc_rto'
]

df_final = df_master[cols_finales].copy()

# 2. Reemplazamos NaNs en pagos acumulados por 0
# (Si es NaN es porque no hubo registros en la tabla pagos, ergo pagó 0)
df_final['total_pagado_acumulado'] = df_final['total_pagado_acumulado'].fillna(0)

# 3. Pequeño ajuste: Si saldo es nulo pero tenemos importe y pagos, lo calculamos
df_final['saldo'] = df_final['saldo'].fillna(df_final['importe_total'] - df_final['total_pagado_acumulado'])

print(f"Dataset depurado. Columnas actuales: {len(df_final.columns)}")

Dataset depurado. Columnas actuales: 21


In [9]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036 entries, 0 to 4035
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      4036 non-null   int64         
 1   fecha_salida            4002 non-null   datetime64[ns]
 2   fecha_devolucion        4003 non-null   datetime64[ns]
 3   fecha_entrada           175 non-null    datetime64[ns]
 4   created_at              4036 non-null   datetime64[ns]
 5   total_dias              3900 non-null   float64       
 6   importe_total           4029 non-null   float64       
 7   subtotal                4035 non-null   float64       
 8   saldo                   4029 non-null   float64       
 9   total_pagos             3370 non-null   float64       
 10  total_pagado_acumulado  4036 non-null   float64       
 11  estado                  4036 non-null   int64         
 12  marca                   4036 non-null   object  

In [10]:
# Diccionarios de mapeo
dict_estados = {
    0: 'Reserva', 1: 'Alquilado', 2: 'Entregado', 3: 'Cancelado',
    4: 'Cerrado', 5: 'Cerrado Pendiente', 6: 'Controlado'
}

dict_clases = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G',
    7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N'
}

# Aplicamos el mapeo creando nuevas columnas
df_final['estado_desc'] = df_final['estado'].map(dict_estados)
df_final['clase_desc'] = df_final['clase'].map(dict_clases)

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036 entries, 0 to 4035
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      4036 non-null   int64         
 1   fecha_salida            4002 non-null   datetime64[ns]
 2   fecha_devolucion        4003 non-null   datetime64[ns]
 3   fecha_entrada           175 non-null    datetime64[ns]
 4   created_at              4036 non-null   datetime64[ns]
 5   total_dias              3900 non-null   float64       
 6   importe_total           4029 non-null   float64       
 7   subtotal                4035 non-null   float64       
 8   saldo                   4029 non-null   float64       
 9   total_pagos             3370 non-null   float64       
 10  total_pagado_acumulado  4036 non-null   float64       
 11  estado                  4036 non-null   int64         
 12  marca                   4036 non-null   object  

In [12]:
# Se observa 136 valores nulos en total_dias. Se recalculan teniendo la fecha de salida y devolucion.
# Además, se eliminan registros imposibles de analizar porque falta información clave.

# 1. Recalcular total_dias donde sea nulo
# Usamos .dt.days para obtener la diferencia entera
df_final['total_dias'] = df_final['total_dias'].fillna(
    (df_final['fecha_devolucion'] - df_final['fecha_salida']).dt.days
)

# 2. Eliminar inconsistencias temporales (Salida > Devolución)
# Solo mantenemos donde la salida es antes o igual a la devolución
df_final = df_final[df_final['fecha_salida'] <= df_final['fecha_devolucion']]

# 3. Eliminar los 34 registros sin fecha_salida (son huérfanos)
df_final = df_final.dropna(subset=['fecha_salida'])


In [14]:
# Consistencia de Kilometrajes
# Un auto no puede volver con menos kilómetros de los que salió. Esto arruinaría tu análisis de desgaste de flota.

# 1. Identificar y limpiar inconsistencias de KM
mask_km_error = df_final['kms_entrada'] < df_final['kms_salida']
# Opción Senior: En lugar de borrar, si el error es pequeño podemos poner NaN para no sesgar
df_final.loc[mask_km_error, 'kms_entrada'] = pd.NA

# 2. Calcular KM recorridos (Feature Engineering preventivo)
df_final['km_recorridos_alquiler'] = df_final['kms_entrada'] - df_final['kms_salida']

In [16]:
# Saneamiento Financiero (total_pagos vs acumulado)
# Como notamos anteriormente, total_pagos tiene muchos nulos, pero total_pagado_acumulado (que calculamos desde la tabla de pagos) es más confiable porque proviene de pagos

# Eliminamos la columna redundante y con nulos
df_final = df_final.drop(columns=['total_pagos'])

# Si el saldo es nulo, lo calculamos como la deuda pendiente real
df_final['saldo'] = df_final['saldo'].fillna(df_final['importe_total'] - df_final['total_pagado_acumulado'])

# Un saldo negativo significa saldo a favor del cliente.
# Para el análisis de mora, nos interesan solo los saldos > 0.


df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4002 entries, 0 to 4035
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      4002 non-null   int64         
 1   fecha_salida            4002 non-null   datetime64[ns]
 2   fecha_devolucion        4002 non-null   datetime64[ns]
 3   fecha_entrada           175 non-null    datetime64[ns]
 4   created_at              4002 non-null   datetime64[ns]
 5   total_dias              4002 non-null   float64       
 6   importe_total           3995 non-null   float64       
 7   subtotal                4002 non-null   float64       
 8   saldo                   3995 non-null   float64       
 9   total_pagado_acumulado  4002 non-null   float64       
 10  estado                  4002 non-null   int64         
 11  marca                   4002 non-null   object        
 12  modelo                  4002 non-null   object       

In [18]:
# Valores duplicados
df_final.duplicated().sum()

np.int64(0)

Se observa que no hay valores duplicados, por lo tanto no se toma ninguna acción en relación a esto.